In [1]:
import os
from tika import parser
import csv

In [2]:
#All pdf files in a dir
filelist = os.listdir('pdfs')

In [3]:
#Extract data from first page, which is the country or state.
def extractBigEntity(textList):
    checkpoint = 0
    statList = []
    for i in range(6):
        stoppoint = textList[checkpoint+1:].index('compared to baseline')
        returnSet = textList[checkpoint:checkpoint+stoppoint+1]
        statList.append(returnSet[-1])
        checkpoint += stoppoint + 1
    return statList, checkpoint

In [4]:
#Extract data from next pages, which is states (for country file) or counties (for state file)
def extractSmallEntity(textList):
    categories = ['Retail & recreation', 'Grocery & pharmacy', 'Parks', 'Transit stations', 'Workplace', 'Residential']
    countyDict = {}
    statList = []
    county = ""
    for i in range(len(textList)):
        if len(statList) == 6 and county != "":
            countyDict[county] = statList
            statList = []
            county = ""
        for element in categories:
            if element in textList[i]:
                if "Retail & recreation" in textList[i]:
                    county = textList[i-1]
                    #print(county)
                if 'compared to baseline' in textList[i+1]:
                    splitLine = textList[i+1].split(' ')
                    #print(splitLine[0])
                    statList.append(splitLine[0])
                    #print(textList[i])
                elif 'Not enough data for this date' in textList[i+1]:
                    statList.append("N/A")
                    #print("Not enough data")
                    #print(textList[i])
    return countyDict

In [24]:
#Create name for csv file with time and country/state name
def extractLocationAndTimeName(filename):
    filenamesplit = filename.split('_')
    if(filenamesplit[2] == 'Mobility'):
        return filenamesplit[0]+'_'+filenamesplit[1]
    else:
        if(os.path.isdir('CSV/'+filenamesplit[1]) == False):
            os.mkdir('CSV/'+filenamesplit[1])
        return filenamesplit[1]+'/'+filenamesplit[0]+'_'+filenamesplit[2]

In [25]:
#Convert a pdf file to csv
def convertPDFtoCSV(filename):
    location = extractLocationAndTimeName(filename)
    
    csvfilename = location+'.csv'
    csvfile = open('CSV/'+csvfilename, 'w')
    writer = csv.writer(csvfile)
    writer.writerow(['X','Retail & recreation', 'Grocery & pharmacy', 'Parks', 'Transit stations', 'Workplace', 'Residential'])
    
    pdf = parser.from_file("pdfs/"+filename)
    text = pdf['content'].strip().split('\n')
    for element in text:
        if element == '':
            text.remove(element)
    overview, checkpoint = extractBigEntity(text)
    overview.insert(0,location)
    writer.writerow(overview)
    resultDict = extractSmallEntity(text[checkpoint:])
    for key in resultDict:
        resultList = resultDict[key]
        resultList.insert(0,key)
        writer.writerow(resultList)
    csvfile.close()

In [26]:
for filename in filelist:
    convertPDFtoCSV(filename)